In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [48]:
# 출력 예쁘게 하기
from rich.console import Console
from rich.table import Table

console = Console()

def rich_docs(docs, title="Retriever Results", max_len=140):
    table = Table(title=title)
    table.add_column("#", justify="right")
    table.add_column("Source")
    table.add_column("Page", justify="right")
    table.add_column("Preview")

    for i, d in enumerate(docs, 1):
        m = d.metadata or {}
        src = (m.get("source","") or "").split("/")[-1]
        page = str(m.get("page_label", m.get("page",0)+1))
        text = (d.page_content or "").strip().replace("\n", " ")
        content = (text[:max_len] + ("…" if len(text) > max_len else ""))
        table.add_row(str(i), src, page, content)

    console.print(table)

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

# 문서 로드하기

In [4]:
from langchain_community.document_loaders import PyPDFLoader
pdf_path = "../data/Samsung_Electronics_Sustainability_Report_2025_KOR.pdf"
loader = PyPDFLoader(pdf_path)
docs = loader.load()

In [5]:
len(docs)

87

In [6]:
docs[2].page_content

'삼성전자 지속가능경영보고서 2025\n03\nOur Company AppendixFacts & Figures PrinciplePlanet People\nOur Company\nCEO 메시지\n회사소개\n기업 지배구조\n중대성 평가\n이해관계자 소통\n04\n05\n06\n07\n09'

# 텍스트 스플리트

In [7]:
# 텍스트 스플리트
from langchain_text_splitters import RecursiveCharacterTextSplitter

spliiter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 100
)
chunck = spliiter.split_documents(docs)

In [8]:
chunck[0].page_content

'삼성전자 지속가능경영보고서 2025\nA Journey  Towards \n a Sustainable Future\nA Journey  Towards\n a Sustainable Future'

# 벡터 스토어 저장 (처음 1회 실행)

In [9]:
emb = OpenAIEmbeddings(model="text-embedding-3-small")
db_path = "../vectorstore/chromadb_advanced_store"
col_name = "samsung"

In [ ]:
# vectorsotre = Chroma.from_documents(
#     documents=chunck,
#     embedding=emb,
#     persist_directory=db_path,
#     collection_name=col_name
# )

In [11]:
# 벡터 저장소 불러오기
load_vectorstore = Chroma(
    persist_directory=db_path,
    collection_name=col_name,
    embedding_function=emb
)

# 1. 키워드 기반 + 기본 검색기 = 하이브리드

## 벡터 검색기(유사도 기반)

In [23]:
ret_similarity = load_vectorstore.as_retriever(
    search_type = "similarity",
    search_kwargs = {"k" : 5}
)

In [24]:
ret_similarity

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x000001E435CAF110>, search_kwargs={'k': 5})

In [ ]:
# 번외 : 메타데이터가 궁금하다

In [17]:
db_docs = load_vectorstore._collection.get(include=["documents", "metadatas"])

In [21]:
db_docs["documents"][0]

'삼성전자 지속가능경영보고서 2025\nA Journey  Towards \n a Sustainable Future\nA Journey  Towards\n a Sustainable Future'

In [22]:
db_docs["metadatas"][0]

{'page_label': '1',
 'creator': 'Adobe InDesign 15.1 (Macintosh)',
 'source': '../data/Samsung_Electronics_Sustainability_Report_2025_KOR.pdf',
 'page': 0,
 'producer': 'Adobe PDF Library 15.0',
 'total_pages': 87,
 'moddate': '2025-09-04T16:51:11+09:00',
 'trapped': '/False',
 'creationdate': '2025-07-10T16:11:16+09:00'}

In [34]:
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever

## 키워드 기반 검색기

In [32]:
from langchain_core.documents import Document

# 저장해둔 vectorsotre를  load_vectorsotre._collection 으로 가져온 뒤
# chunck와 동일한 형태인 Document 리스트로 만들기

bm_doc = []

for content, meta, id in zip(db_docs["documents"], db_docs["metadatas"], db_docs["ids"]):
    bm_doc.append(Document(page_content=content, metadata=meta, id=id))

In [33]:
bm_doc[0]

Document(id='89bf31e4-51dc-48b9-939f-9fe43bd34d22', metadata={'page_label': '1', 'creator': 'Adobe InDesign 15.1 (Macintosh)', 'source': '../data/Samsung_Electronics_Sustainability_Report_2025_KOR.pdf', 'page': 0, 'producer': 'Adobe PDF Library 15.0', 'total_pages': 87, 'moddate': '2025-09-04T16:51:11+09:00', 'trapped': '/False', 'creationdate': '2025-07-10T16:11:16+09:00'}, page_content='삼성전자 지속가능경영보고서 2025\nA Journey  Towards \n a Sustainable Future\nA Journey  Towards\n a Sustainable Future')

In [36]:
bm25 = BM25Retriever.from_documents(bm_doc) # 키워드 기반 검색기
bm25.k = 5

## 벡터 검색기 + bm25 = 하이브리드 검색기

In [37]:
ret_hybrid = EnsembleRetriever(
    retrievers=[ret_similarity, bm25],
    weights=[0.7, 0.3]
)

In [38]:
question = "삼성전자의 2025년 전망은?"
result = ret_hybrid.invoke(question)
rich_docs(result, title = "하이브리드 검색기")

                                                 하이브리드 검색기                                                 
┏━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ # ┃ Source                                           ┃ Page ┃ Preview                                           ┃
┡━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 1 │ Samsung_Electronics_Sustainability_Report_2025_… │   86 │ 삼성전자 지속가능경영보고서 2025 86               │
│   │                                                  │      │ 삼성전자주식회사는 경제·사회·환경적 가치 창출     │
│   │                                                  │      │ 성과를 다양한 이해관계자와 투명하게 소통하기 위 … │
│   │                                                  │      │ 2025년 열여덟 번째 지속가능경영보고서를           │
│   │                                                  │      │ 발간합니다. 작성 기준 본 보고서는 지속가능경영    │
│   │                                                  │      │ 보고 기준인 GRI(G…                                │
│ 2 │ Samsung_Electronics_Sustainability_Report_2025_… │    1 │ 삼성전자 지속가능경영보고서 2025 A Journey        │
│   │                                                  │      │ Towards   a Sustainable Future A Journey  Towards │
│   │                                                  │      │ a Sustainable Future                              │
│ 3 │ Samsung_Electronics_Sustainability_Report_2025_… │    4 │ 삼성전자 지속가능경영보고서 2025 04 Our Company   │
│   │                                                  │      │ AppendixFacts & Figures PrinciplePlanet People    │
│   │                                                  │      │ 주주, 고객, 협력회사, 그리고 임직원 여러분,       │
│   │                                                  │      │ 2024년은 글로벌 지정학적 리스크와 AI 기술의 성장  │
│   │                                                  │      │ …                                                 │
│ 4 │ Samsung_Electronics_Sustainability_Report_2025_… │   77 │ 삼성전자 지속가능경영보고서 2025 77 Scope 1, 2    │
│   │                                                  │      │ 온실가스 배출량 검증 의견서 Our Company           │
│   │                                                  │      │ AppendixFacts & Figures PrinciplePlanet People    │
│ 5 │ Samsung_Electronics_Sustainability_Report_2025_… │   76 │ 삼성전자 지속가능경영보고서 2025 76 독립된        │
│   │                                                  │      │ 인증인의 인증보고서 Our Company AppendixFacts &   │
│   │                                                  │      │ Figures PrinciplePlanet People                    │
│ 6 │ Samsung_Electronics_Sustainability_Report_2025_… │    6 │ 삼성전자 지속가능경영보고서 2025 06 Our Company   │
│   │                                                  │      │ AppendixFacts & Figures PrinciplePlanet People    │
│   │                                                  │      │ 주요 분야별 협의회  환경     [DX부문] 환경안전    │
│   │                                                  │      │ 회의, ESG공시TF  [DS부문]  탄소감축위원…          │
│ 7 │ Samsung_Electronics_Sustainability_Report_2025_… │   86 │ ·  삼성전자주식회사 지속가능경영 웹사이트         │
│   │                                                  │      │ http://www.samsung.com/sec/sustainability/main ·  │
│   │                                                  │      │ 삼성전자주식회사 IR 웹사이트                      │
│   │                                                  │      │ http://www.samsung.com/sec/ir ·  삼성전자주식회 … │
│   │                                                  │      │ 뉴…                                               │
│ 8 │ Samsung_Electronics_Sustainability_Report_2025_… │   42 │ 삼성전자 지속가능경영보고서 2025 42 여성인력 활 … │
│   │                                                  │      │ 강화 여성 리더십 목표제 운영 삼성전자는 여성      │
│   │                                                  │      │ 임원의 비중을 2022년 6.9% 대비 2030년 까지 2배    │
│   │                                                  │      │ 이상 확대하고자 하는 목표를 가지고 여성 리더의    │
│   │                                                  │      │ 양적·질적 확

# 2. 압축 검색기 (Compression retreiver)
- 검색된 문서가 길 때 -> llm 을 이용해서 내용을 압축
- 문서 내용이 너무 파편화 되어 있는 경우 -> 압축 진행 -> 찌꺼기 제거(의미 없는 엔터, 헤더, 풋터, 기호 등)
- 비용 문제 -> 각 문서별로 전부 압축을 진행해서 사용할 경우 비용 문제 발생

### 2-1. 기본 압축 검색기

In [40]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor, LLMChainFilter
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    temperature=0,
    model = "gpt-4.1-mini"
)

In [41]:
# 압축기 생성
compressor = LLMChainFilter.from_llm(model)

# 압축 검색기 생성 = 유사도 검색 -> 문서 내용 압축
ret_compressor = ContextualCompressionRetriever(
    base_retriever = ret_similarity,
    base_compressor = compressor
)

In [44]:
question = "삼성 전자의 목표와 년도"
com_result = ret_compressor.invoke(question)

In [47]:
rich_docs(com_result, title = "압축기 검색기 사용")

                                                압축기 검색기 사용                                                 
┏━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ # ┃ Source                                           ┃ Page ┃ Preview                                           ┃
┡━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 1 │ Samsung_Electronics_Sustainability_Report_2025_… │    4 │ 삼성전자 지속가능경영보고서 2025 04 Our Company   │
│   │                                                  │      │ AppendixFacts & Figures PrinciplePlanet People    │
│   │                                                  │      │ 주주, 고객, 협력회사, 그리고 임직원 여러분,       │
│   │                                                  │      │ 2024년은 글로벌 지정학적 리스크와 AI 기술의 성장  │
│   │                                                  │      │ …                                                 │
│ 2 │ Samsung_Electronics_Sustainability_Report_2025_… │    1 │ 삼성전자 지속가능경영보고서 2025 A Journey        │
│   │                                                  │      │ Towards   a Sustainable Future A Journey  Towards │
│   │                                                  │      │ a Sustainable Future                              │
└───┴──────────────────────────────────────────────────┴──────┴───────────────────────────────────────────────────┘

### 2-2. 임베딩 기반 경량 압축 (비용 x)

In [53]:
from langchain.retrievers.document_compressors import EmbeddingsFilter

ret_mmr = load_vectorstore.as_retriever(
    search_type = "mmr",
    search_kwargs={"k": 5, "fetch_k": 20, "lambda_mult": 0.25},
)
emb_filter = EmbeddingsFilter(
    embeddings=emb,
    similarity_threshold=0.2
)
comp_embed = ContextualCompressionRetriever(
    base_retriever=ret_mmr,
    base_compressor=emb_filter
)
comp_embed_result = comp_embed.invoke(question)

In [54]:
rich_docs(comp_embed_result, title='임베딩 기반 필터기법')

                                               임베딩 기반 필터기법                                                
┏━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ # ┃ Source                                           ┃ Page ┃ Preview                                           ┃
┡━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 1 │ Samsung_Electronics_Sustainability_Report_2025_… │    4 │ 삼성전자 지속가능경영보고서 2025 04 Our Company   │
│   │                                                  │      │ AppendixFacts & Figures PrinciplePlanet People    │
│   │                                                  │      │ 주주, 고객, 협력회사, 그리고 임직원 여러분,       │
│   │                                                  │      │ 2024년은 글로벌 지정학적 리스크와 AI 기술의 성장  │
│   │                                                  │      │ …                                                 │
│ 2 │ Samsung_Electronics_Sustainability_Report_2025_… │    1 │ 삼성전자 지속가능경영보고서 2025 A Journey        │
│   │                                                  │      │ Towards   a Sustainable Future A Journey  Towards │
│   │                                                  │      │ a Sustainable Future                              │
│ 3 │ Samsung_Electronics_Sustainability_Report_2025_… │   86 │ ·  삼성전자주식회사 지속가능경영 웹사이트         │
│   │                                                  │      │ http://www.samsung.com/sec/sustainability/main ·  │
│   │                                                  │      │ 삼성전자주식회사 IR 웹사이트                      │
│   │                                                  │      │ http://www.samsung.com/sec/ir ·  삼성전자주식회 … │
│   │                                                  │      │ 뉴…                                               │
│ 4 │ Samsung_Electronics_Sustainability_Report_2025_… │   40 │ 삼성전자 지속가능경영보고서 2025 40 활동 결사의   │
│   │                                                  │      │ 자유와 단체교섭 보장 결사의 자 유는 근로자가      │
│   │                                                  │      │ 노동조합을 결성하 거나 가 입할 수 있는            │
│   │                                                  │      │ 권리입니다. 단체교섭은 사용자와 노동조합이 건     │
│   │                                                  │      │ 설적인 논의를 통해  근로조건을 확립하고 근로자의  │
│   │                                                  │      │ 기회균등을 …                                      │
│ 5 │ Samsung_Electronics_Sustainability_Report_2025_… │   57 │ 삼성전자 지속가능경영보고서 2025 57 고객 서비스   │
│   │                                                  │      │ 삼성전자는 최선의 서비스를 제공하여 고객 만족도 … │
│   │                                                  │      │ 극대화 하는 것을  목표로 고객의 요구사항에        │
│   │                                                  │      │ 신속하고 효과적으로 대응합니다. 제품 서비스  진 … │
│   │                                                  │      │ 현황을 실시간으로 관리할 수 있는 시스 템을 구축…  │
└───┴──────────────────────────────────────────────────┴──────┴───────────────────────────────────────────────────┘

# 3. 리랭커(reranker)

In [ ]:
# uv add sentence-transformers
# uv add langchain_huggingface

- 검색기로 후보군 추출(10-30) (너무 적은 갯수는 리랭커 할 이유가 없음)

In [86]:
ret_similarity_10 = load_vectorstore.as_retriever(
    search_type = "similarity",
    search_kwargs = {"k" : 11}
)
question = "삼성과 현대 중에서 어느 기업이 더 나아?"
ret_similarity_result = ret_similarity_10.invoke(question)
rich_docs(ret_similarity_result, title='기업 비교')

                                                     기업 비교                                                     
┏━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃  # ┃ Source                                           ┃ Page ┃ Preview                                          ┃
┡━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│  1 │ Samsung_Electronics_Sustainability_Report_2025_… │    5 │ 삼성전자 지속가능경영보고서 2025 05 Our Company  │
│    │                                                  │      │ AppendixFacts & Figures PrinciplePlanet People   │
│    │                                                  │      │ 회사소개 About Us 삼성전자주식회사(이하          │
│    │                                                  │      │ 삼성전자)는 인재와 기술을 기반으로 최고의 제품 … │
│    │                                                  │      │ 서비스를…                                        │
│  2 │ Samsung_Electronics_Sustainability_Report_2025_… │    3 │ 삼성전자 지속가능경영보고서 2025 03 Our Company  │
│    │                                                  │      │ AppendixFacts & Figures PrinciplePlanet People   │
│    │                                                  │      │ Our Company CEO 메시지 회사소개 기업 지배구조    │
│    │                                                  │      │ 중대성 평가 이해관계자 소통 04 05 06 …           │
│  3 │ Samsung_Electronics_Sustainability_Report_2025_… │    4 │ 삼성전자 지속가능경영보고서 2025 04 Our Company  │
│    │                                                  │      │ AppendixFacts & Figures PrinciplePlanet People   │
│    │                                                  │      │ 주주, 고객, 협력회사, 그리고 임직원 여러분,      │
│    │                                                  │      │ 2024년은 글로벌 지정학적 리스크와 AI 기술의 성 … │
│    │                                                  │      │ …                                                │
│  4 │ Samsung_Electronics_Sustainability_Report_2025_… │   46 │ 정기적으로 모니터링하고 시스템으로 관리합니다.   │
│    │                                                  │      │ 하위 공급망 관리 지속적으로 증가하는 공급망      │
│    │                                                  │      │ 변동성과 불 확실성에 대응하기 위하여 삼성전자는  │
│    │                                                  │      │ 1 차 협력회사뿐 아니라 주요  품목을 공급하는     │
│    │                                                  │      │ 하위 협력회사에 대한 정보 관리를 점진적으로      │
│    │                                                  │      │ 확대하고 있습니다.…                              │
│  5 │ Samsung_Electronics_Sustainability_Report_2025_… │   50 │ 삼성전자 지속가능경영보고서 2025 50 안전보건     │
│    │                                                  │      │ 삼성전자는 DX부문의 Global EHS실장, DS부문의     │
│    │                                                  │      │ 글로벌 제조&인프라  총괄장인 CSO(Chief Safety    │
│    │                                                  │      │ Officer)를 중심으로 2030년까지 상주  협력회사    │
│    │                                                  │      │ 안전보건 역량과 관리체계 …                       │
│  6 │ Samsung_Electronics_Sustainability_Report_2025_… │   54 │ 삼성전자 지속가능경영보고서 2025 54 Bespoke AI   │
│    │                                                  │      │ 로봇청소기 보안 인증 획득 2024년에는 Bespoke AI  │
│    │                                                  │      │ 스팀 로봇청소기가 KISA 개인정보보호중심설계      │
│    │                                                  │      │ (PbD, Privacy by Design) 인증과 KISA IoT 보안    │
│    │                                                  │      │ 인증 중 …                                        │
│  7 │ Samsung_Electronics_Sustainability_Report_2025_… │   45 │ 협력회사를 지원하고 교육합니다.  협력회사의      │
│    │                                                  │      │ 지속가능경영 관리 수준 향상을 위한 실사와 개     │
│    │                                                  │      │ 선조치 이행, 정보공개 등의 성과는 해 당 부서의   │
│    │                                                  │      │ 경영진과 실무자들의 KPI로 반영되며 목표 달성은   │
│    │                            

In [87]:
from langchain_community.cross_encoders.huggingface import HuggingFaceCrossEncoder
from langchain.retrievers.document_compressors import CrossEncoderReranker

In [88]:
hf_ce = HuggingFaceCrossEncoder(
    model_name = "cross-encoder/ms-marco-MiniLM-L6-v2",
    model_kwargs = {
        "device" : "cuda",
        "max_length" : 512
    }
)

In [ ]:
# uv add hf_xet

In [89]:
compressor = CrossEncoderReranker(
    model = hf_ce,
    top_n = 11
)

ret_reranker = ContextualCompressionRetriever(
    base_retriever=ret_similarity_10,
    base_compressor=compressor,
)

rerank_result = ret_reranker.invoke(question)

In [90]:
rich_docs(rerank_result, title="리랭크 결과", max_len=30)

                                                    리랭크 결과                                                    
┏━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃  # ┃ Source                                           ┃ Page ┃ Preview                                          ┃
┡━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│  1 │ Samsung_Electronics_Sustainability_Report_2025_… │   45 │ 삼성전자 지속가능경영보고서 2025 45 협력회사     │
│    │                                                  │      │ 행동…                                            │
│  2 │ Samsung_Electronics_Sustainability_Report_2025_… │   46 │ 정기적으로 모니터링하고 시스템으로 관리합니다.   │
│    │                                                  │      │ 하위 공…                                         │
│  3 │ Samsung_Electronics_Sustainability_Report_2025_… │   45 │ 협력회사를 지원하고 교육합니다.  협력회사의      │
│    │                                                  │      │ 지속가능경…                                      │
│  4 │ Samsung_Electronics_Sustainability_Report_2025_… │   40 │ 스타일에 맞춰 혜택을 받을 수 있는 선택적         │
│    │                                                  │      │ 복리후생 제…                                     │
│  5 │ Samsung_Electronics_Sustainability_Report_2025_… │    5 │ 삼성전자 지속가능경영보고서 2025 05 Our Com…     │
│  6 │ Samsung_Electronics_Sustainability_Report_2025_… │   54 │ 삼성전자 지속가능경영보고서 2025 54 Bespoke…     │
│  7 │ Samsung_Electronics_Sustainability_Report_2025_… │    4 │ 삼성전자 지속가능경영보고서 2025 04 Our Com…     │
│  8 │ Samsung_Electronics_Sustainability_Report_2025_… │   50 │ 삼성전자 지속가능경영보고서 2025 50 안전보건     │
│    │                                                  │      │ 삼성…                                            │
│  9 │ Samsung_Electronics_Sustainability_Report_2025_… │    4 │ 재활용 소재를 적용하는 성과를 이뤘습니다.        │
│    │                                                  │      │ DS(Dev…                                          │
│ 10 │ Samsung_Electronics_Sustainability_Report_2025_… │   61 │ 삼성전자 지속가능경영보고서 2025 61 Facts &…     │
│ 11 │ Samsung_Electronics_Sustainability_Report_2025_… │    3 │ 삼성전자 지속가능경영보고서 2025 03 Our Com…     │
└────┴──────────────────────────────────────────────────┴──────┴──────────────────────────────────────────────────┘

# 4. 리오더(Reorder)
- 리랭크와 같이 사용
- 리랭크에서는 맥락(내용의 흐름) 고려 x

In [ ]:
from langchain_community.document_transformers import LongContextReorder

reorder = LongContextReorder()
reorderd_result = reorder.transform_documents(rerank_result)

# reorderd_result = []
# rerank_result.reverse()
# for i, value in enumerate(rerank_result):
#     if i % 2 == 0:
#         reorderd_result.append(value)
#     else:
#         reorderd_result.insert(0, value)

In [92]:
rich_docs(reorderd_result, "리오더 결과", 30)

                                                    리오더 결과                                                    
┏━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃  # ┃ Source                                           ┃ Page ┃ Preview                                          ┃
┡━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│  1 │ Samsung_Electronics_Sustainability_Report_2025_… │   45 │ 삼성전자 지속가능경영보고서 2025 45 협력회사     │
│    │                                                  │      │ 행동…                                            │
│  2 │ Samsung_Electronics_Sustainability_Report_2025_… │   45 │ 협력회사를 지원하고 교육합니다.  협력회사의      │
│    │                                                  │      │ 지속가능경…                                      │
│  3 │ Samsung_Electronics_Sustainability_Report_2025_… │    5 │ 삼성전자 지속가능경영보고서 2025 05 Our Com…     │
│  4 │ Samsung_Electronics_Sustainability_Report_2025_… │    4 │ 삼성전자 지속가능경영보고서 2025 04 Our Com…     │
│  5 │ Samsung_Electronics_Sustainability_Report_2025_… │    4 │ 재활용 소재를 적용하는 성과를 이뤘습니다.        │
│    │                                                  │      │ DS(Dev…                                          │
│  6 │ Samsung_Electronics_Sustainability_Report_2025_… │    3 │ 삼성전자 지속가능경영보고서 2025 03 Our Com…     │
│  7 │ Samsung_Electronics_Sustainability_Report_2025_… │   61 │ 삼성전자 지속가능경영보고서 2025 61 Facts &…     │
│  8 │ Samsung_Electronics_Sustainability_Report_2025_… │   50 │ 삼성전자 지속가능경영보고서 2025 50 안전보건     │
│    │                                                  │      │ 삼성…                                            │
│  9 │ Samsung_Electronics_Sustainability_Report_2025_… │   54 │ 삼성전자 지속가능경영보고서 2025 54 Bespoke…     │
│ 10 │ Samsung_Electronics_Sustainability_Report_2025_… │   40 │ 스타일에 맞춰 혜택을 받을 수 있는 선택적         │
│    │                                                  │      │ 복리후생 제…                                     │
│ 11 │ Samsung_Electronics_Sustainability_Report_2025_… │   46 │ 정기적으로 모니터링하고 시스템으로 관리합니다.   │
│    │                                                  │      │ 하위 공…                                         │
└────┴──────────────────────────────────────────────────┴──────┴──────────────────────────────────────────────────┘